In [1]:
import pandas as pd
import sqlite3 as sq
import duckdb
from utils.quack import Quack

In [2]:
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',]
cols2roll = ["completions", "attempts", "carries", "passing_yards", "rushing_yards", "receiving_yards"]

In [3]:
weekly = Quack.fetch_table('weekly')
per_game = weekly.groupby(['game_id','season','week','team', 'opponent_team'])[stat_cols].sum().reset_index().sort_values(by=['season','week'])
per_game['pass_pct'] = per_game['attempts'] / (per_game['attempts'] + per_game['carries'])
per_game['yards_per_carry'] = per_game['rushing_yards'] / per_game['carries']
per_game['yards_per_reception'] = per_game['receiving_yards'] / per_game['completions']
cols2roll += ['yards_per_carry', 'yards_per_reception', 'pass_pct']

In [4]:
roll_stats = [x + '_team_roll' for x in cols2roll]
print(roll_stats)
team = per_game.copy()
team[roll_stats] = team.groupby("team")[
    cols2roll
].transform(lambda x: x.rolling(7, min_periods=1).mean()).sort_values(by='completions')
team[roll_stats] = team[roll_stats] / team[roll_stats].mean()
team[['team','season','week'] + roll_stats]

['completions_team_roll', 'attempts_team_roll', 'carries_team_roll', 'passing_yards_team_roll', 'rushing_yards_team_roll', 'receiving_yards_team_roll', 'yards_per_carry_team_roll', 'yards_per_reception_team_roll', 'pass_pct_team_roll']


,team,season,week,completions_team_roll,attempts_team_roll,carries_team_roll,passing_yards_team_roll,rushing_yards_team_roll,receiving_yards_team_roll,yards_per_carry_team_roll,yards_per_reception_team_roll,pass_pct_team_roll
0,ARI,2000,1,1.354012,1.466331,0.743322,1.353715,0.378948,1.353734,0.518308,0.987572,1.284719
1,NYG,2000,1,0.822079,0.748128,1.523811,0.732198,1.965242,0.732208,1.311204,0.879791,0.685263
2,BAL,2000,1,0.870436,0.957604,1.337980,0.847136,1.233784,0.847148,0.937508,0.961347,0.851339
3,PIT,2000,1,0.822079,1.167080,0.668990,0.847136,0.264382,0.847148,0.401789,1.017897,1.237801
4,CAR,2000,1,0.822079,0.778053,0.743322,0.779025,0.987027,0.779036,1.350011,0.936056,1.022532
...,...,...,...,...,...,...,...,...,...,...,...,...
13077,TB,2025,7,1.084591,1.047380,0.929153,1.074580,0.877498,1.074595,0.942352,0.994947,1.049742
13078,DAL,2025,7,1.298746,1.128605,0.929153,1.168841,1.076414,1.168858,1.154474,0.913151,1.082568
13079,WAS,2025,7,0.856620,0.872104,1.019414,0.886057,1.311840,0.886069,1.281162,1.047687,0.927961
13080,LAC,2025,8,1.236572,1.137155,0.945081,1.134786,1.126772,1.134801,1.312924,0.916059,1.083087


In [5]:
opp_stats = [x + '_opp_roll' for x in cols2roll]
opp = per_game.copy()
opp[opp_stats] = opp.groupby("opponent_team")[
    cols2roll
].transform(lambda x: x.rolling(7, min_periods=1).mean()).sort_values(by='completions')
opp[opp_stats] = opp[opp_stats] / opp[opp_stats].mean()
opp[['opponent_team','season','week'] + opp_stats]

,opponent_team,season,week,completions_opp_roll,attempts_opp_roll,carries_opp_roll,passing_yards_opp_roll,rushing_yards_opp_roll,receiving_yards_opp_roll,yards_per_carry_opp_roll,yards_per_reception_opp_roll,pass_pct_opp_roll
0,NYG,2000,1,1.354780,1.467218,0.743824,1.354438,0.379147,1.354433,0.518094,0.987752,1.284087
1,ARI,2000,1,0.822545,0.748581,1.524839,0.732589,1.966273,0.732587,1.310664,0.879950,0.684926
2,PIT,2000,1,0.870930,0.958183,1.338883,0.847589,1.234431,0.847586,0.937121,0.961522,0.850920
3,BAL,2000,1,0.822545,1.167786,0.669442,0.847589,0.264521,0.847586,0.401623,1.018082,1.237192
4,WAS,2000,1,0.822545,0.778524,0.743824,0.779441,0.987545,0.779438,1.349455,0.936226,1.022029
...,...,...,...,...,...,...,...,...,...,...,...,...
13077,DET,2025,7,1.036821,1.018070,0.823519,0.991795,0.773409,0.991791,0.966437,0.963967,1.086875
13078,WAS,2025,7,0.953876,0.919685,1.078545,1.087932,1.110988,1.087928,1.050393,1.141894,0.931482
13079,DAL,2025,7,1.133591,1.018070,1.121049,1.179201,1.245768,1.179197,1.133625,1.041082,0.952531
13080,MIN,2025,8,0.877842,0.795634,1.131675,0.868885,1.150037,0.868882,0.976055,0.976981,0.850626


In [6]:
team[['game_id','team','season','week'] + roll_stats].to_csv('data/agg/team_stats.csv')
opp[['game_id','opponent_team','season','week'] + opp_stats].to_csv('data/agg/opp_stats.csv')

In [7]:

con = duckdb.connect("data/nfl.duckdb")

team = team[['game_id','team','season','week'] + roll_stats]
team.to_csv('data/agg/team_stats.csv')
opp = opp[['game_id','opponent_team','season','week'] + opp_stats]
opp.to_csv('data/agg/opp_stats.csv')

con.execute("CREATE OR REPLACE TABLE team_feats AS SELECT * FROM team")
con.execute("CREATE OR REPLACE TABLE opp_feats AS SELECT * FROM opp")

con.commit()
con.close()

In [8]:
team.columns

Index(['game_id', 'team', 'season', 'week', 'completions_team_roll',
       'attempts_team_roll', 'carries_team_roll', 'passing_yards_team_roll',
       'rushing_yards_team_roll', 'receiving_yards_team_roll',
       'yards_per_carry_team_roll', 'yards_per_reception_team_roll',
       'pass_pct_team_roll'],
      dtype='object')

In [9]:
opp

,game_id,opponent_team,season,week,completions_opp_roll,attempts_opp_roll,carries_opp_roll,passing_yards_opp_roll,rushing_yards_opp_roll,receiving_yards_opp_roll,yards_per_carry_opp_roll,yards_per_reception_opp_roll,pass_pct_opp_roll
0,2000_01_ARI_NYG,NYG,2000,1,1.354780,1.467218,0.743824,1.354438,0.379147,1.354433,0.518094,0.987752,1.284087
1,2000_01_ARI_NYG,ARI,2000,1,0.822545,0.748581,1.524839,0.732589,1.966273,0.732587,1.310664,0.879950,0.684926
2,2000_01_BAL_PIT,PIT,2000,1,0.870930,0.958183,1.338883,0.847589,1.234431,0.847586,0.937121,0.961522,0.850920
3,2000_01_BAL_PIT,BAL,2000,1,0.822545,1.167786,0.669442,0.847589,0.264521,0.847586,0.401623,1.018082,1.237192
4,2000_01_CAR_WAS,WAS,2000,1,0.822545,0.778524,0.743824,0.779441,0.987545,0.779438,1.349455,0.936226,1.022029
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13077,2025_07_TB_DET,DET,2025,7,1.036821,1.018070,0.823519,0.991795,0.773409,0.991791,0.966437,0.963967,1.086875
13078,2025_07_WAS_DAL,WAS,2025,7,0.953876,0.919685,1.078545,1.087932,1.110988,1.087928,1.050393,1.141894,0.931482
13079,2025_07_WAS_DAL,DAL,2025,7,1.133591,1.018070,1.121049,1.179201,1.245768,1.179197,1.133625,1.041082,0.952531
13080,2025_08_MIN_LAC,MIN,2025,8,0.877842,0.795634,1.131675,0.868885,1.150037,0.868882,0.976055,0.976981,0.850626


In [10]:
team

,game_id,team,season,week,completions_team_roll,attempts_team_roll,carries_team_roll,passing_yards_team_roll,rushing_yards_team_roll,receiving_yards_team_roll,yards_per_carry_team_roll,yards_per_reception_team_roll,pass_pct_team_roll
0,2000_01_ARI_NYG,ARI,2000,1,1.354012,1.466331,0.743322,1.353715,0.378948,1.353734,0.518308,0.987572,1.284719
1,2000_01_ARI_NYG,NYG,2000,1,0.822079,0.748128,1.523811,0.732198,1.965242,0.732208,1.311204,0.879791,0.685263
2,2000_01_BAL_PIT,BAL,2000,1,0.870436,0.957604,1.337980,0.847136,1.233784,0.847148,0.937508,0.961347,0.851339
3,2000_01_BAL_PIT,PIT,2000,1,0.822079,1.167080,0.668990,0.847136,0.264382,0.847148,0.401789,1.017897,1.237801
4,2000_01_CAR_WAS,CAR,2000,1,0.822079,0.778053,0.743322,0.779025,0.987027,0.779036,1.350011,0.936056,1.022532
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13077,2025_07_TB_DET,TB,2025,7,1.084591,1.047380,0.929153,1.074580,0.877498,1.074595,0.942352,0.994947,1.049742
13078,2025_07_WAS_DAL,DAL,2025,7,1.298746,1.128605,0.929153,1.168841,1.076414,1.168858,1.154474,0.913151,1.082568
13079,2025_07_WAS_DAL,WAS,2025,7,0.856620,0.872104,1.019414,0.886057,1.311840,0.886069,1.281162,1.047687,0.927961
13080,2025_08_MIN_LAC,LAC,2025,8,1.236572,1.137155,0.945081,1.134786,1.126772,1.134801,1.312924,0.916059,1.083087


In [11]:
roll_stats

['completions_team_roll',
 'attempts_team_roll',
 'carries_team_roll',
 'passing_yards_team_roll',
 'rushing_yards_team_roll',
 'receiving_yards_team_roll',
 'yards_per_carry_team_roll',
 'yards_per_reception_team_roll',
 'pass_pct_team_roll']